# Inter-Reliability Dataset Formatter

## Updates
2/6/2018: Created a empty dataset for calculating ground truth/inter-rater reliability called interrater_2017_12_12_upwork_analysis_unitedstates_allskills.csv

In [5]:
import httplib2
import oauth2
import urllib3
import types
import re
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from gender_detector import GenderDetector 
import psycopg2, psycopg2.extras
from causalinference import CausalModel
from causalinference.utils import random_data
import httplib
import base64
import json # For Microsoft Face API
import urllib as urllib # For Microsoft Face API
import time 
import csv
import datetime 
import random
import os.path

class UpworkDataFormatter:
    
    # Select random number from user count 
    # Check that we were able to download the photo, the data exists, education history is not empty 
    # Format the link to the photo 
    # Format the education description (NOT DONE YET)
    # Write to CSV 
    
    def __init__(self):
        # Settings
        self.interrater_data_file_name = './csv_files/interrater_2017_12_12_upwork_analysis_unitedstates_allskills_2.csv'
        self.original_data_file_name = './csv_files/2017_12_12_upwork_analysis_unitedstates_allskills.csv' # Filename for all data
        self.original_face_data_file_name = './csv_files/2017_12_12_upwork_face_data_unitedstates_allskills.csv' # Filename for gender data
        self.resized_photo_path = './resized_profile_photos/resized_images_unitedstates_allskills_2017_12_12/'
        self.github_photo_path = 'https://raw.githubusercontent.com/efoongch/upwork-pay-by-gender/master/resized_profile_photos/resized_images_unitedstates_allskills_2017_12_12/'
        
        self.total_dataset_size = 55518 # insert dataset size here
        self.interrater_dataset_size = 200 # insert size of dataset for interrater reliablity
        self.random_users_list = []
        self.random_worker_id_list = []
        
        # Initialize arrays for Causal Analysis 
        self.users_w_complete_data = []
        
        # Connect to the database 
        self.conn = psycopg2.connect("dbname=eureka01")
        self.cur = self.conn.cursor(cursor_factory=psycopg2.extras.DictCursor)
        psycopg2.extensions.register_adapter(dict, psycopg2.extras.Json)
        
        # Get detailed_info from workers in our database
        self.cur.execute("SELECT detailed_info FROM upwork_unitedstates_allskills_2017_12_12;")        
    
    def find_users_w_complete_data(self):
        count_thru_database = 1
        
        for user in self.cur:
            if (user is not None and user != {}):
                self.users_w_complete_data.append(count_thru_database)
            count_thru_database += 1
        
    def save_to_csv(self):
        df = pd.read_csv(self.original_data_file_name)
        
        # Create an array of random users
        while len(self.random_users_list) < self.interrater_dataset_size + 1: 
            random_number = random.randint(1, self.total_dataset_size)
    
            # If the file exists and there is no existing number in the dataset, then add it to the array 
            filename = self.resized_photo_path + str(random_number) + ".jpg"
            if (os.path.isfile(filename)): 
                if not random_number in self.random_users_list:
                    if random_number in self.users_w_complete_data:
                        if (df.iloc[random_number-1]['worker_id'] != "error"):
                            if (df.iloc[random_number-1]['degree'] != "" and df.iloc[random_number-1]['degree'] != "Error: No education"): # Make sure there is education history
                                self.random_users_list.append(random_number)
        
        print "This is the list of random users we obtained: {0}".format(self.random_users_list)
        
        # Find the relevant worker ids
        for worker in self.random_users_list:
            worker_id = df.iloc[worker-1]['worker_id']
            print worker_id
            self.random_worker_id_list.append(worker_id)
        
        print "This is the list of random worker ids: {0}".format(self.random_worker_id_list)
        
        # Start writing the csv file 
        with open(self.interrater_data_file_name, 'w') as csvfile:
            print "We have started a new csv file!"
            fieldnames = ['user_count', 'worker_id', 'photo_link', 'gender', 'age_range', 'full_education', 'education']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            
            count = 1
            for worker in self.random_worker_id_list:
                print "Now writing data for random user: {0}".format(count)
                user_count = self.random_users_list[count-1]
                worker_id = self.random_worker_id_list[count-1]
                photo_path = self.github_photo_path + str(self.random_users_list[count-1]) + ".jpg"
                formatted_photo_path = '=IMAGE("' + photo_path + '", 1)'# Formatted for Google Sheets
                full_education = self.return_education(worker) 
                
                writer.writerow({'user_count': user_count, 'worker_id': worker_id, 'photo_link': formatted_photo_path, 'gender': "", 
                                 'age_range': "", 'full_education': full_education, 'education': ""})
                count += 1
                
    def return_education(self, worker_id):
        print "We are in return education"
        self.cur.execute("""SELECT detailed_info FROM upwork_unitedstates_allskills_2017_12_12 WHERE user_id = %s;""", (worker_id,))
        for user in self.cur:
            try:
                return user[0]["education"]["institution"]
            except Exception as error:
                print "Error here: {0}".format(error)
                print user[0]
                break 


myObject = UpworkDataFormatter()
myObject.find_users_w_complete_data()
myObject.save_to_csv()

This is the list of random users we obtained: [22033, 43163, 33867, 39317, 54612, 39888, 8881, 19369, 20175, 26419, 30939, 25680, 52806, 43497, 3953, 1942, 46161, 19306, 27765, 47853, 45584, 48544, 36201, 16965, 882, 31082, 19051, 16837, 25831, 18481, 33235, 9310, 9080, 25192, 24296, 13134, 18670, 51178, 25404, 5789, 51608, 13628, 24390, 34388, 30879, 38792, 8749, 53204, 40284, 35556, 3698, 14586, 38540, 34578, 24218, 24532, 22478, 37086, 34611, 16599, 37728, 34620, 43325, 9851, 38357, 13157, 45119, 54959, 8924, 18286, 35698, 7375, 6549, 21458, 38183, 4419, 9707, 16936, 48846, 47847, 88, 38823, 40796, 29281, 30236, 9807, 12101, 12456, 2076, 40005, 5486, 50036, 52870, 14883, 34792, 45967, 4013, 45499, 36272, 35469, 43059, 37928, 12717, 22472, 9775, 12713, 51121, 48581, 21479, 5331, 25357, 40678, 19317, 18408, 13569, 25928, 19517, 19324, 38549, 54962, 11534, 44267, 53513, 35039, 22435, 35173, 732, 24965, 15392, 46059, 52019, 15071, 50216, 48680, 50394, 51485, 41275, 32001, 52336, 14869, 

AttributeError: UpworkDataFormatter instance has no attribute 'random_work_id_list'